In [43]:
import bagpy
import pandas as pd
import sys
import numpy as np
import os
from scipy.spatial.transform import Rotation as R

conversion = 0.18
file_path = '/home/shc/Desktop/data/1030/drill_exp_3.bag'
b = bagpy.bagreader(file_path)
b.topic_table

[INFO]  Data folder /home/shc/Desktop/data/1030/drill_exp_3 already exists. Not creating.


,Topics,Types,Message Count,Frequency
0,/fwd_limage/compressed,sensor_msgs/CompressedImage,5886,29.901647
1,/fwd_pose_camhand,geometry_msgs/PoseStamped,5886,29.903565
2,/fwd_pose_drill,geometry_msgs/PoseStamped,5886,29.902073
3,/fwd_pose_pan,geometry_msgs/PoseStamped,5886,29.896532
4,/fwd_rimage/compressed,sensor_msgs/CompressedImage,5887,29.895146


In [44]:
pose_cam_data = b.message_by_topic('/fwd_pose_camhand')
pose_pan_data = b.message_by_topic('/fwd_pose_pan')

df_pan = pd.read_csv(pose_pan_data)[:8449]
df_cam_hand = pd.read_csv(pose_cam_data)[:8449]

In [45]:
os.chdir('/home/shc/RoboMaster/util')
from Solver import solver
from dataLoader import dataLoader
sol = solver()
ld = dataLoader()

phacon2pan = np.load('/home/shc/RoboMaster/params/phacon2pan_1028.npy')
# pan2phacon = np.load('/home/shc/RoboMaster/params/phacon2pan.npy')

X = np.load("/home/shc/Desktop/data/1030/handeye_good/hand_eye_X.npy")

In [46]:
def getRealPose(idx, pd_data):
    df = pd_data
    pose_x = df['pose.position.x'][idx]*1000
    pose_y = df['pose.position.y'][idx]*1000
    pose_z = df['pose.position.z'][idx]*1000
    orin_x = df['pose.orientation.x'][idx]
    orin_y = df['pose.orientation.y'][idx]
    orin_z = df['pose.orientation.z'][idx]
    orin_w = df['pose.orientation.w'][idx]
    pose = np.array([pose_x, pose_y, pose_z, orin_x, orin_y, orin_z, orin_w])
    # print(real_pose, '\n', pose)
    return pose

In [47]:
num_frames = len(df_cam_hand)
pose_pan_data = np.zeros([num_frames, 4, 4])
pose_camhand_data = np.zeros([num_frames, 4, 4])
for i in range(num_frames):
    quaternion_pan = getRealPose(i, df_pan)
    quaternion_camhand = getRealPose(i, df_cam_hand)
    _, op2pan = sol.seven2trans(quaternion_pan)
    _, op2camhand = sol.seven2trans(quaternion_camhand)
    
    pan2phacon = sol.invTransformation(phacon2pan)
    op2phacon = op2pan@pan2phacon
    op2cam = op2camhand@X
     ## F_cv_ambf, cam2ambf
    extrinsic = np.array([[0, 1, 0, 0], [0, 0, -1, 0],
                          [-1, 0, 0, 0], [0, 0, 0, 1]])
    # op2cam = op2cam@extrinsic
    cam2phacon = sol.invTransformation(op2cam)@op2phacon
print(cam2phacon)
# r = R.from_matrix(cam2phacon[:3,:3])
# r.as_euler('xyz', degrees=True)


[[  0.83892      0.14436571   0.52475879   5.48181556]
 [ -0.4179353   -0.44674004   0.79104578  -2.29320368]
 [  0.34863065  -0.88293934  -0.31444362 142.70541997]
 [  0.           0.           0.           1.        ]]


In [48]:
ambf2phacon = sol.invTransformation(extrinsic)@cam2phacon

In [49]:
t = ambf2phacon[:3, 3]/conversion/1000
r = R.from_matrix(ambf2phacon[:3,:3])
Rot = r.as_euler('xyz')
# Rot = Rot@Rot_offset
print('position: {{x: {0}, y: {1}, z: {2}}}'.format(t[0],t[1],t[2]))
print('orientation: {{r: {0}, p: {1}, y: {2}}}'.format(Rot[0], Rot[1], Rot[2]))

position: {x: -0.7928078887099471, y: 0.030454530873993867, z: 0.012740020446863001}
orientation: {r: 2.627498577341158, p: -0.4311714218293079, y: 1.9646533206271166}
